In [8]:
from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd
import os

app = Dash(__name__)

#reading csv file

# Get the absolute path to the data folder
data_folder = os.path.abspath('data')

# Construct the absolute path to the CSV file

# Read the CSV file
df_0 = pd.read_csv(os.path.join(data_folder, 'daily_sales_data_0.csv'))


#filtering only pink morsel
df_0 = df_0[df_0['product'] == 'pink morsel']

# from price column removing $ and converting to number
df_0['price'] = pd.to_numeric(df_0['price'].str.replace('$', '', regex=False), errors='coerce') 

#adding new column named sales where sales=price*quantity
df_0['sales'] = df_0['price'] * df_0['quantity']

#dropping the columns
columns_to_drop = ['price', 'quantity']
df_0 = df_0.drop(columns=columns_to_drop)

df_1 = pd.read_csv(os.path.join(data_folder, 'daily_sales_data_1.csv'))

df_1 = df_1[df_1['product'] == 'pink morsel']

df_1['price'] = pd.to_numeric(df_1['price'].str.replace('$', '', regex=False), errors='coerce')

df_1['sales'] = df_1['price'] * df_1['quantity']

columns_to_drop = ['price', 'quantity']
df_1 = df_1.drop(columns=columns_to_drop)

df_2 = pd.read_csv(os.path.join(data_folder, 'daily_sales_data_2.csv'))

df_2 = df_2[df_2['product'] == 'pink morsel']

df_2['price'] = pd.to_numeric(df_2['price'].str.replace('$', '', regex=False), errors='coerce')

df_2['sales'] = df_2['price'] * df_2['quantity']

columns_to_drop = ['price', 'quantity']
df_2 = df_2.drop(columns=columns_to_drop)

# combining all three different datasets into single dataset
combined_df = pd.concat([df_0,df_1,df_2],ignore_index = True)
combined_df = combined_df.drop(columns = 'product')

# sorting dataset by region 
c_north = combined_df[combined_df['region'] == 'north']
c_south = combined_df[combined_df['region'] == 'south']
c_east = combined_df[combined_df['region'] == 'east']
c_west = combined_df[combined_df['region'] == 'west']

# thresold value for north region where threshold=sales on Jan 15, 2021
sales_on_query_data_north = c_north.loc[c_north['date']== '2021-01-15','sales'] 
north_range=sales_on_query_data_north.values[0]  if not sales_on_query_data_north.empty else 0

line_color = ['sales greater than thresold' if y > north_range else 'sales lesser than thresold' for y in c_north['sales']]
c_north.loc[:, 'Color'] = line_color

# thresold value for south region where threshold=sales on Jan 15, 2021
sales_on_query_data_south = c_south.loc[c_south['date']== '2021-01-15','sales'] 
south_range=sales_on_query_data_south.values[0]  if not sales_on_query_data_south.empty else 0
line_color = ['sales greater than thresold' if y > south_range else 'sales lesser than thresold' for y in c_south['sales']]
c_south.loc[:, 'Color'] = line_color

# thresold value for east region where threshold=sales on Jan 15, 2021
sales_on_query_data_east = c_east.loc[c_east['date']== '2021-01-15','sales'] 
east_range=sales_on_query_data_east.values[0]  if not sales_on_query_data_east.empty else 0
line_color = ['sales greater than thresold' if y > east_range else 'sales lesser than thresold' for y in c_east['sales']]
c_east.loc[:, 'Color'] = line_color

# thresold value for west region where threshold=sales on Jan 15, 2021
sales_on_query_data_west = c_west.loc[c_west['date']== '2021-01-15','sales'] 
west_range=sales_on_query_data_west.values[0]  if not sales_on_query_data_west.empty else 0
line_color = ['sales greater than thresold' if y > west_range else 'sales lesser than thresold' for y in c_west['sales']]
c_west.loc[:, 'Color'] = line_color

app.layout = html.Div(children=[
    html.H1(
        children='Soul Foods',
        style={
            'textAlign': 'center'
        }
    ),

    html.Div(children='Sales of Pink Morsel', style={
        'textAlign': 'center',
        'fontSize': '30px'
    }),
    
    html.Div(children='Sales = Price*Quantity',style={
        'border':'2px solid black',
        'margin-right':'800px'
    }),

    dcc.Graph(
        id='line chart 1',
        figure=px.line(c_north, x='date', y='sales', color='Color',title='North region: Sales on each day')
    ),
    dcc.Graph(
        id='line chart 2',
        figure=px.line(c_south, x='date', y='sales', color='Color',title='South region: Sales on each day')
    ),
    dcc.Graph(
        id='line chart 3',
        figure=px.line(c_east, x='date', y='sales', color='Color',title='East region: Sales on each day')
    ),
    dcc.Graph(
        id='line chart 4',
        figure=px.line (c_west, x='date', y='sales', color='Color',title='West region: Sales on each day')
    )
])

if __name__ == '__main__':
    app.run(debug=True)
